In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas import Series, DataFrame
from datetime import datetime

In [2]:
url_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

df_confirmed_primary = pd.read_csv(url_confirmed)
df_deaths_primary = pd.read_csv(url_deaths)


In [3]:
date_columns = df_confirmed_primary.columns[4:]
date_columns

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '2/28/23', '3/1/23', '3/2/23', '3/3/23', '3/4/23', '3/5/23', '3/6/23',
       '3/7/23', '3/8/23', '3/9/23'],
      dtype='object', length=1143)

In [4]:
# setting Date Format
date_format = pd.to_datetime(date_columns)
date_format

DatetimeIndex(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
               '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
               '2020-01-30', '2020-01-31',
               ...
               '2023-02-28', '2023-03-01', '2023-03-02', '2023-03-03',
               '2023-03-04', '2023-03-05', '2023-03-06', '2023-03-07',
               '2023-03-08', '2023-03-09'],
              dtype='datetime64[ns]', length=1143, freq=None)

In [5]:
# setting Date Range
range = (date_format >= '2020-02-01') & (date_format <= '2021-07-31')

In [6]:
df_confirmed = df_confirmed_primary.iloc[:, :4].join(df_confirmed_primary.iloc[:, 4:][date_columns[range]])
df_deaths = df_deaths_primary.iloc[:, :4].join(df_deaths_primary.iloc[:, 4:][date_columns[range]])

In [7]:
# Sum of confirmed cases and deaths
global_confirmed = df_confirmed.iloc[:, 4:].sum(axis=0)
global_deaths = df_deaths.iloc[:, 4:].sum(axis=0)